In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dense, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
import numpy as np

In [2]:
# list devices for tensorflow
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [8]:
IMAGE_SIZE = (256, 256)
BATCH_SIZE = 32  
DIRECTORY = "datasets"

In [9]:
train_dataset = image_dataset_from_directory(
    DIRECTORY,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE)

validation_dataset = image_dataset_from_directory(
    DIRECTORY,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE)

class_names = train_dataset.class_names
num_classes = len(class_names)


Found 1178 files belonging to 4 classes.
Using 943 files for training.
Found 1178 files belonging to 4 classes.
Using 235 files for validation.


In [10]:
class_names

['Alternaria_Leaf_Spot', 'Bacterial_Spot', 'Healthy_Petchay', 'Undefined']

In [11]:
base_model = MobileNetV3Large(input_shape=(*IMAGE_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False  

In [12]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x) 

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 256, 256, 3)          0         ['input_1[0][0]']             
                                                                                                  
 Conv (Conv2D)               (None, 128, 128, 16)         432       ['rescaling[0][0]']           
                                                                                                  
 Conv/BatchNorm (BatchNorma  (None, 128, 128, 16)         64        ['Conv[0][0]']                
 lization)                                                                                    

 expanded_conv_2/depthwise   (None, 64, 64, 72)           648       ['re_lu_4[0][0]']             
 (DepthwiseConv2D)                                                                                
                                                                                                  
 expanded_conv_2/depthwise/  (None, 64, 64, 72)           288       ['expanded_conv_2/depthwise[0]
 BatchNorm (BatchNormalizat                                         [0]']                         
 ion)                                                                                             
                                                                                                  
 re_lu_5 (ReLU)              (None, 64, 64, 72)           0         ['expanded_conv_2/depthwise/Ba
                                                                    tchNorm[0][0]']               
                                                                                                  
 expanded_

                                                                                                  
 re_lu_9 (ReLU)              (None, 32, 32, 120)          0         ['expanded_conv_4/expand/Batch
                                                                    Norm[0][0]']                  
                                                                                                  
 expanded_conv_4/depthwise   (None, 32, 32, 120)          3000      ['re_lu_9[0][0]']             
 (DepthwiseConv2D)                                                                                
                                                                                                  
 expanded_conv_4/depthwise/  (None, 32, 32, 120)          480       ['expanded_conv_4/depthwise[0]
 BatchNorm (BatchNormalizat                                         [0]']                         
 ion)                                                                                             
          

                                                                                                  
 tf.__operators__.add_3 (TF  (None, 1, 1, 120)            0         ['expanded_conv_5/squeeze_exci
 OpLambda)                                                          te/Conv_1[0][0]']             
                                                                                                  
 re_lu_14 (ReLU)             (None, 1, 1, 120)            0         ['tf.__operators__.add_3[0][0]
                                                                    ']                            
                                                                                                  
 tf.math.multiply_3 (TFOpLa  (None, 1, 1, 120)            0         ['re_lu_14[0][0]']            
 mbda)                                                                                            
                                                                                                  
 expanded_

                                                                                                  
 re_lu_17 (ReLU)             (None, 16, 16, 200)          0         ['tf.__operators__.add_6[0][0]
                                                                    ']                            
                                                                                                  
 tf.math.multiply_6 (TFOpLa  (None, 16, 16, 200)          0         ['re_lu_17[0][0]']            
 mbda)                                                                                            
                                                                                                  
 multiply_3 (Multiply)       (None, 16, 16, 200)          0         ['expanded_conv_7/expand/Batch
                                                                    Norm[0][0]',                  
                                                                     'tf.math.multiply_6[0][0]']  
          

...

  (DepthwiseConv2D)                                                                               
                                                                                                  
 expanded_conv_10/depthwise  (None, 16, 16, 480)          1920      ['expanded_conv_10/depthwise[0
 /BatchNorm (BatchNormaliza                                         ][0]']                        
 tion)                                                                                            
                                                                                                  
 tf.__operators__.add_13 (T  (None, 16, 16, 480)          0         ['expanded_conv_10/depthwise/B
 FOpLambda)                                                         atchNorm[0][0]']              
                                                                                                  
 re_lu_24 (ReLU)             (None, 16, 16, 480)          0         ['tf.__operators__.add_13[0][0
          

                                                                                                  
 tf.math.multiply_16 (TFOpL  (None, 16, 16, 672)          0         ['re_lu_27[0][0]']            
 ambda)                                                                                           
                                                                                                  
 multiply_12 (Multiply)      (None, 16, 16, 672)          0         ['expanded_conv_11/depthwise/B
                                                                    atchNorm[0][0]',              
                                                                     'tf.math.multiply_16[0][0]'] 
                                                                                                  
 expanded_conv_11/squeeze_e  (None, 1, 1, 672)            0         ['multiply_12[0][0]']         
 xcite/AvgPool (GlobalAvera                                                                       
 gePooling

                                                                                                  
 multiply_14 (Multiply)      (None, 8, 8, 672)            0         ['expanded_conv_12/depthwise/B
                                                                    atchNorm[0][0]',              
                                                                     'tf.math.multiply_19[0][0]'] 
                                                                                                  
 expanded_conv_12/squeeze_e  (None, 1, 1, 672)            0         ['multiply_14[0][0]']         
 xcite/AvgPool (GlobalAvera                                                                       
 gePooling2D)                                                                                     
                                                                                                  
 expanded_conv_12/squeeze_e  (None, 1, 1, 168)            113064    ['expanded_conv_12/squeeze_exc
 xcite/Con

                                                                                                  
 expanded_conv_13/squeeze_e  (None, 1, 1, 240)            0         ['expanded_conv_13/squeeze_exc
 xcite/Relu (ReLU)                                                  ite/Conv[0][0]']              
                                                                                                  
 expanded_conv_13/squeeze_e  (None, 1, 1, 960)            231360    ['expanded_conv_13/squeeze_exc
 xcite/Conv_1 (Conv2D)                                              ite/Relu[0][0]']              
                                                                                                  
 tf.__operators__.add_23 (T  (None, 1, 1, 960)            0         ['expanded_conv_13/squeeze_exc
 FOpLambda)                                                         ite/Conv_1[0][0]']            
                                                                                                  
 re_lu_34 

                                                                                                  
 tf.__operators__.add_26 (T  (None, 1, 1, 960)            0         ['expanded_conv_14/squeeze_exc
 FOpLambda)                                                         ite/Conv_1[0][0]']            
                                                                                                  
 re_lu_37 (ReLU)             (None, 1, 1, 960)            0         ['tf.__operators__.add_26[0][0
                                                                    ]']                           
                                                                                                  
 tf.math.multiply_26 (TFOpL  (None, 1, 1, 960)            0         ['re_lu_37[0][0]']            
 ambda)                                                                                           
                                                                                                  
 expanded_

In [15]:
print(len(model.layers))

266


In [16]:
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=validation_dataset)

Epoch 1/10


30/30 [==============================] - 18s 485ms/step - loss: 0.4647 - accuracy: 0.8452 - val_loss: 0.1029 - val_accuracy: 0.9830
Epoch 2/10
30/30 [==============================] - 13s 426ms/step - loss: 0.0715 - accuracy: 0.9852 - val_loss: 0.0476 - val_accuracy: 0.9957
Epoch 3/10
30/30 [==============================] - 13s 422ms/step - loss: 0.0387 - accuracy: 0.9958 - val_loss: 0.0317 - val_accuracy: 0.9957
Epoch 4/10
30/30 [==============================] - 13s 425ms/step - loss: 0.0274 - accuracy: 0.9979 - val_loss: 0.0235 - val_accuracy: 0.9957
Epoch 5/10
30/30 [==============================] - 13s 416ms/step - loss: 0.0191 - accuracy: 0.9989 - val_loss: 0.0180 - val_accuracy: 0.9957
Epoch 6/10
30/30 [==============================] - 13s 438ms/step - loss: 0.0149 - accuracy: 1.0000 - val_loss: 0.0145 - val_accuracy: 1.0000
Epoch 7/10
30/30 [==============================] - 13s 430ms/step - loss: 0.0118 - accuracy: 1.0000 - val_loss: 0.0125 - val_accuracy: 0.99

In [17]:
base_model.trainable = True

fine_tune_at = 100

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

model.compile(loss='sparse_categorical_crossentropy',
              optimizer = Adam(learning_rate=1e-5),
              metrics=['accuracy'])

history_fine = model.fit(train_dataset,
                         epochs=1,
                         validation_data=validation_dataset)


30/30 [==============================] - 35s 901ms/step - loss: 0.9160 - accuracy: 0.6405 - val_loss: 0.0098 - val_accuracy: 1.0000


In [20]:
img_path ="C:/Users/skywa/notebook3/datasets/Bacterial_Spot/001005.jpg"

img = tf.keras.utils.load_img(
    img_path, target_size= IMAGE_SIZE
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

In [21]:
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions)

print(
    "This image most likely belongs to {} ."
    .format(class_names[np.argmax(score)])
)

1/1 [==============================] - 1s 966ms/step
This image most likely belongs to Bacterial_Spot .


In [22]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 256, 256, 3)          0         ['input_1[0][0]']             
                                                                                                  
 Conv (Conv2D)               (None, 128, 128, 16)         432       ['rescaling[0][0]']           
                                                                                                  
 Conv/BatchNorm (BatchNorma  (None, 128, 128, 16)         64        ['Conv[0][0]']                
 lization)                                                                                    

 expanded_conv_2/depthwise   (None, 64, 64, 72)           648       ['re_lu_4[0][0]']             
 (DepthwiseConv2D)                                                                                
                                                                                                  
 expanded_conv_2/depthwise/  (None, 64, 64, 72)           288       ['expanded_conv_2/depthwise[0]
 BatchNorm (BatchNormalizat                                         [0]']                         
 ion)                                                                                             
                                                                                                  
 re_lu_5 (ReLU)              (None, 64, 64, 72)           0         ['expanded_conv_2/depthwise/Ba
                                                                    tchNorm[0][0]']               
                                                                                                  
 expanded_

                                                                                                  
 re_lu_9 (ReLU)              (None, 32, 32, 120)          0         ['expanded_conv_4/expand/Batch
                                                                    Norm[0][0]']                  
                                                                                                  
 expanded_conv_4/depthwise   (None, 32, 32, 120)          3000      ['re_lu_9[0][0]']             
 (DepthwiseConv2D)                                                                                
                                                                                                  
 expanded_conv_4/depthwise/  (None, 32, 32, 120)          480       ['expanded_conv_4/depthwise[0]
 BatchNorm (BatchNormalizat                                         [0]']                         
 ion)                                                                                             
          

                                                                                                  
 tf.__operators__.add_3 (TF  (None, 1, 1, 120)            0         ['expanded_conv_5/squeeze_exci
 OpLambda)                                                          te/Conv_1[0][0]']             
                                                                                                  
 re_lu_14 (ReLU)             (None, 1, 1, 120)            0         ['tf.__operators__.add_3[0][0]
                                                                    ']                            
                                                                                                  
 tf.math.multiply_3 (TFOpLa  (None, 1, 1, 120)            0         ['re_lu_14[0][0]']            
 mbda)                                                                                            
                                                                                                  
 expanded_

                                                                                                  
 re_lu_17 (ReLU)             (None, 16, 16, 200)          0         ['tf.__operators__.add_6[0][0]
                                                                    ']                            
                                                                                                  
 tf.math.multiply_6 (TFOpLa  (None, 16, 16, 200)          0         ['re_lu_17[0][0]']            
 mbda)                                                                                            
                                                                                                  
 multiply_3 (Multiply)       (None, 16, 16, 200)          0         ['expanded_conv_7/expand/Batch
                                                                    Norm[0][0]',                  
                                                                     'tf.math.multiply_6[0][0]']  
          

...

  (DepthwiseConv2D)                                                                               
                                                                                                  
 expanded_conv_10/depthwise  (None, 16, 16, 480)          1920      ['expanded_conv_10/depthwise[0
 /BatchNorm (BatchNormaliza                                         ][0]']                        
 tion)                                                                                            
                                                                                                  
 tf.__operators__.add_13 (T  (None, 16, 16, 480)          0         ['expanded_conv_10/depthwise/B
 FOpLambda)                                                         atchNorm[0][0]']              
                                                                                                  
 re_lu_24 (ReLU)             (None, 16, 16, 480)          0         ['tf.__operators__.add_13[0][0
          

                                                                                                  
 tf.math.multiply_16 (TFOpL  (None, 16, 16, 672)          0         ['re_lu_27[0][0]']            
 ambda)                                                                                           
                                                                                                  
 multiply_12 (Multiply)      (None, 16, 16, 672)          0         ['expanded_conv_11/depthwise/B
                                                                    atchNorm[0][0]',              
                                                                     'tf.math.multiply_16[0][0]'] 
                                                                                                  
 expanded_conv_11/squeeze_e  (None, 1, 1, 672)            0         ['multiply_12[0][0]']         
 xcite/AvgPool (GlobalAvera                                                                       
 gePooling

                                                                                                  
 multiply_14 (Multiply)      (None, 8, 8, 672)            0         ['expanded_conv_12/depthwise/B
                                                                    atchNorm[0][0]',              
                                                                     'tf.math.multiply_19[0][0]'] 
                                                                                                  
 expanded_conv_12/squeeze_e  (None, 1, 1, 672)            0         ['multiply_14[0][0]']         
 xcite/AvgPool (GlobalAvera                                                                       
 gePooling2D)                                                                                     
                                                                                                  
 expanded_conv_12/squeeze_e  (None, 1, 1, 168)            113064    ['expanded_conv_12/squeeze_exc
 xcite/Con

                                                                                                  
 expanded_conv_13/squeeze_e  (None, 1, 1, 240)            0         ['expanded_conv_13/squeeze_exc
 xcite/Relu (ReLU)                                                  ite/Conv[0][0]']              
                                                                                                  
 expanded_conv_13/squeeze_e  (None, 1, 1, 960)            231360    ['expanded_conv_13/squeeze_exc
 xcite/Conv_1 (Conv2D)                                              ite/Relu[0][0]']              
                                                                                                  
 tf.__operators__.add_23 (T  (None, 1, 1, 960)            0         ['expanded_conv_13/squeeze_exc
 FOpLambda)                                                         ite/Conv_1[0][0]']            
                                                                                                  
 re_lu_34 

                                                                                                  
 tf.__operators__.add_26 (T  (None, 1, 1, 960)            0         ['expanded_conv_14/squeeze_exc
 FOpLambda)                                                         ite/Conv_1[0][0]']            
                                                                                                  
 re_lu_37 (ReLU)             (None, 1, 1, 960)            0         ['tf.__operators__.add_26[0][0
                                                                    ]']                           
                                                                                                  
 tf.math.multiply_26 (TFOpL  (None, 1, 1, 960)            0         ['re_lu_37[0][0]']            
 ambda)                                                                                           
                                                                                                  
 expanded_

In [23]:
model.save('C:/Users/skywa/notebook3/')

INFO:tensorflow:Assets written to: C:/Users/skywa/notebook3/assets


INFO:tensorflow:Assets written to: C:/Users/skywa/notebook3/assets


In [24]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('bokchoymodel.tflite', 'wb') as f:
  f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\skywa\AppData\Local\Temp\tmpl7ut8ib5\assets


INFO:tensorflow:Assets written to: C:\Users\skywa\AppData\Local\Temp\tmpl7ut8ib5\assets


In [25]:
class_labels = class_names  # Replace with your actual class labels

# Write the class labels to a labels.txt file
with open("petchay_labels.txt", "w") as f:
    for label in class_labels:
        f.write(label + "\n")